In [1]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy

import matplotlib.pyplot as plt
import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.FCGF as FCGF

from utils.config import Config
from scipy.ndimage import gaussian_filter1d

In [26]:
dataset_dir = "data/raw_data"
features_dir = "data/features"
experiment = "exp_12"
calib_period = 4

In [22]:
for trial in os.listdir(os.path.join(dataset_dir, experiment)):
    
    if not trial.startswith("trial"):
        continue
    
    for subject in os.listdir(os.path.join(dataset_dir, experiment, trial, "secondary")):
        for sequence in os.listdir(os.path.join(dataset_dir, experiment, trial, "secondary", subject)):
            print(f"Processing > {trial} {sequence}")
            motion_dir = os.path.join(dataset_dir, experiment, trial, "secondary", subject, sequence, "motion")
            sequence_dir = os.path.join(dataset_dir, experiment, trial, "secondary", subject, sequence, "frames")
            feature_dir = os.path.join(features_dir, experiment, trial, "0.03", subject, sequence)

            sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")

            accel_df = pd.read_csv(os.path.join(motion_dir, "accel.csv"))
            gyro_df = pd.read_csv(os.path.join(motion_dir, "gyro.csv"))

            accel_df.drop_duplicates("timestamp", inplace=True)
            gyro_df.drop_duplicates("timestamp", inplace=True)
            imu_df = pd.merge(accel_df, gyro_df, on="timestamp", suffixes=("a", "g"))

            frame_rate = accel_df.shape[0] / (accel_df.timestamp.values[-1] - accel_df.timestamp.values[0]) * 1000
            win_len = int(frame_rate * calib_period)

            # remove first row as the dt is 0
            imu_df = imu_df.iloc[1:]
            # reset index in pandas data frame
            imu_df.reset_index(drop=True, inplace=True)

            imu_df = imu_df.iloc[win_len:].copy()

            # load ground truth trajectory
            start_t = np.where(sequence_ts == functions.nearest(sequence_ts, imu_df.timestamp.values[0]))[0][0]

            sequence_ts = sequence_ts[start_t:]
            for t in sequence_ts[:start_t]:
                if os.path.exists(os.path.join(sequence_dir, f"frame-{t}.depth.png")):
                    os.remove(os.path.join(sequence_dir, f"frame-{t}.depth.png"))
                if os.path.exists(os.path.join(sequence_dir, f"frame-{t}.color.png")):
                    os.remove(os.path.join(sequence_dir, f"frame-{t}.color.png"))
                if os.path.exists(os.path.join(feature_dir, f"{t}.global.npz")):
                    os.remove(os.path.join(feature_dir, f"{t}.global.npz"))
                if os.path.exists(os.path.join(feature_dir, f"{t}.secondary.npz")):
                    os.remove(os.path.join(feature_dir, f"{t}.secondary.npz"))    

01
